In [ ]:
# 프로젝트 시작
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
# 2013년부터 2022년까지 데이터 가져오기
for season in range(2013, 2023):
  url = f'https://www.transfermarkt.co.uk/premier-league/tabelle/wettbewerb/GB1?saison_id={season}'
  driver.get(url)

  if season == 2013:  # 팝업창 닫기
    iframe = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//iframe[@id='sp_message_iframe_953778']"))
        )
    driver.switch_to.frame(iframe)
        # "Accept & continue" 버튼에 대한 대기
    accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept & continue')]"))
        )
    accept_button.click()


  driver.execute_script("return navigator.userAgent")
  sleep(3)
  # 필요한 정보 추출
  team_rows = driver.find_elements(By.CSS_SELECTOR, '.responsive-table tbody tr')
  team_rows

  team_data = []

  for i in range(1,21):
      team_point = driver.find_element(By.CSS_SELECTOR, f'#yw1 > table > tbody > tr:nth-child({i}) > td:nth-child(10)').text # 승점
      team_name_element = driver.find_element(By.CSS_SELECTOR, f'#yw1 > table > tbody > tr:nth-child({i}) > td.no-border-links.hauptlink > a') # 팀 명
      team_name = team_name_element.get_attribute('title') # 팀 명 'title'값으로 받아오기
      team_rank = driver.find_element(By.CSS_SELECTOR, f'#yw1 > table > tbody > tr:nth-child({i}) > td:nth-child(1)').text # 순위
      team_score = driver.find_element(By.CSS_SELECTOR, f'#yw1 > table > tbody > tr:nth-child({i}) > td:nth-child(8)').text.split(':')[0] # 득점
      team_lose_score = driver.find_element(By.CSS_SELECTOR, f'#yw1 > table > tbody > tr:nth-child({i}) > td:nth-child(8)').text.split(':')[1] # 실점
      team_data.append({'Team': team_name,'Rank':team_rank, 'Point': team_point, 'Score': team_score, 'Lose_score': team_lose_score})
  # 데이터프레임 생성
  df = pd.DataFrame(team_data)
  # CSV 파일로 저장
  filename = f'{season}-{season+1} 프리미어리그 순위표.csv'
  df.to_csv(filename, index=False)
  print(f'{filename} 파일이 생성되었습니다.')
